In [2]:
import requests
import json
import schedule
import time
import datetime
from dotenv import load_dotenv
import os
import yfinance as yf
from pymongo import MongoClient
import requests_cache
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

load_dotenv()
uri = os.getenv("MONGO_URL")
client = MongoClient(uri)
db = client["yahoo_finance"]

PROXY_SERVER=os.getenv("PROXY_SERVER")

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'moneybot/1.0'


### Implement Rate Limiting, Request Caching, and Proxy Server to Prevent Blacklisting

In [6]:
file_path = "Colab Notebooks/company_tickers.json"

def read_company_tickers(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

company_tickers = read_company_tickers(file_path)
tickers = [item.get("ticker") for item in company_tickers.values()]


### Get Tickers

In [ ]:
def fetch_and_store_ticker_data(n1, n2):
    yf_col = db["tickers"]
    batch_start_time = time.time()
    for t in tickers[n1:n2]:
        try:
            df = yf.download(t, period="max")
            if df.empty:
                print(f"No data found for {t}, possible delisting")
                continue
            data_dict = df.reset_index().to_dict(orient='records')
            
            yf_col.insert_many(data_dict)
        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                    print("429 error detected, delaying to respect rate limit...")
                time.sleep(1.0)

    batch_end_time = time.time()
    batch_total_time = batch_end_time - batch_start_time
    print(f"Batch ran for {batch_total_time:.2f} seconds")

In [ ]:
# start_time = time.time()
# num_batches = len(tickers/500)
# n1, n2 = 0, 499 
# for b in range(0,num_batches):
#     fetch_and_store_ticker_data(n1, n2)
#     n1 += 500    
#     n2 += 500    
# end_time = time.time()
# total_time = end_time - start_time
# print(f"Program ran for {total_time:.2f} seconds")

### Get Ticker

In [46]:
def fetch_and_store_history_data(db, n1, n2):
    yf_col = db["tickers"]
    sleep_time = 2.5
    batch_start_time = time.time()
    for t in tickers[n1:n2]:
        try:
            df = yf.Ticker(t).history(
               period="max",
               prepost=True,
               actions=True,
               proxy=PROXY_SERVER,
               keepna=True
            )
            if df.empty:
                print(f"No data found for {t}, possible delisting")
                continue
            data_dict = df.reset_index().to_dict(orient='records')
            
            for record in data_dict:
                ts = str(record['Date']).replace('-', '')
                record['_id'] = f'{t}_{ts[:8]}'  # Generate a unique ID for each record
            
            yf_col.insert_many(data_dict)
        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                if sleep_time<3.0:
                    sleep_time += 0.5
                    print("429 error detected, delaying to respect rate limit...")
                time.sleep(10.0)
        time.sleep(sleep_time)
    batch_end_time = time.time()
    batch_total_time = batch_end_time - batch_start_time
    print(f"Batch ran for {batch_total_time:.2f} seconds")

In [47]:
start_time = time.time()
num_batches = (len(tickers) // 500) + 1
# num_batches = 1
n1, n2 = 0, 499 
for b in range(0, num_batches):
    fetch_and_store_history_data(db, n1, n2)
    n1 += 500    
    n2 += 500    
end_time = time.time()
total_time = end_time - start_time
print(f"Program ran for {total_time:.2f} seconds")


Batch ran for 1347.77 seconds


TMIN: No timezone found, symbol may be delisted


No data found for TMIN, possible delisting


PMVC: No timezone found, symbol may be delisted


No data found for PMVC, possible delisting
Batch ran for 1346.01 seconds


...

No data found for TTNPW, possible delisting


MDLVY: No timezone found, symbol may be delisted


No data found for MDLVY, possible delisting


SAAYY: No timezone found, symbol may be delisted


No data found for SAAYY, possible delisting
Batch ran for 892.38 seconds
Program ran for 27825.08 seconds


### Get Actions

In [15]:
def fetch_and_store_actions_data(db):
    yf_col = db["actions"]
    sleep_time = 2.5
    for t in tickers:
        try:
            df = yf.Ticker(t).get_actions(
               proxy=PROXY_SERVER
            )
            if df.empty:
                print(f"No data found for {t}, possible delisting")
                continue
            data_dict = df.reset_index().to_dict(orient='records')
            
            for record in data_dict:
                ts = str(record['Date']).replace('-', '')
                record['_id'] = f'{t}_ACTN_{ts[:8]}'  # Generate a unique ID for each record
            
            yf_col.insert_many(data_dict)
        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                if sleep_time<3.0:
                    sleep_time += 0.5
                    print("429 error detected, delaying to respect rate limit...")
                time.sleep(10.0)
        time.sleep(sleep_time)

In [16]:
fetch_and_store_actions_data(db)


No data found for PDD, possible delisting
No data found for NOW, possible delisting
No data found for UBER, possible delisting
No data found for ARM, possible delisting
No data found for ABNB, possible delisting
No data found for REGN, possible delisting
No data found for CMG, possible delisting
No data found for PYPL, possible delisting
No data found for CRWD, possible delisting
No data found for WDAY, possible delisting
No data found for SPOT, possible delisting
No data found for GLD, possible delisting
No data found for DASH, possible delisting
No data found for SNOW, possible delisting
No data found for NU, possible delisting
No data found for SMCI, possible delisting
No data found for COIN, possible delisting
No data found for PLTR, possible delisting
No data found for MFG, possible delisting
No data found for SQ, possible delisting
No data found for TEAM, possible delisting
No data found for MRNA, possible delisting
No data found for DDOG, possible delisting
No data found for IQV

JMKJ: No timezone found, symbol may be delisted


No data found for JMKJ, possible delisting
No data found for ABCL, possible delisting
No data found for VITL, possible delisting
No data found for TH, possible delisting
No data found for CARS, possible delisting
No data found for PRTA, possible delisting
No data found for CIFR, possible delisting
No data found for GOOS, possible delisting
No data found for THR, possible delisting
No data found for AMRC, possible delisting
No data found for ANGI, possible delisting
No data found for CRON, possible delisting
No data found for ENVX, possible delisting
No data found for BV, possible delisting
No data found for BLBD, possible delisting
No data found for GENI, possible delisting
No data found for TDCX, possible delisting
No data found for CSIQ, possible delisting
No data found for CNTA, possible delisting
No data found for ZIP, possible delisting
No data found for TASK, possible delisting
No data found for NN, possible delisting
No data found for SHCO, possible delisting
No data found for I

ETH: No timezone found, symbol may be delisted


No data found for ETH, possible delisting
No data found for VREX, possible delisting
No data found for AMBC, possible delisting
No data found for ANAB, possible delisting
No data found for TMCI, possible delisting
No data found for BTDR, possible delisting
No data found for AZUL, possible delisting
No data found for LAZR, possible delisting
No data found for KYTX, possible delisting
No data found for SVA, possible delisting
No data found for PNTG, possible delisting
No data found for EXK, possible delisting
No data found for SVRA, possible delisting
No data found for DNTH, possible delisting
No data found for AOSL, possible delisting
No data found for VMEO, possible delisting
No data found for NBBK, possible delisting
No data found for HKD, possible delisting
No data found for SOC, possible delisting
No data found for PGRU, possible delisting
No data found for BH-A, possible delisting
No data found for EXAI, possible delisting
No data found for BLND, possible delisting
No data found fo

...

No data found for TTNPW, possible delisting
No data found for CMCAU, possible delisting
No data found for CMCAW, possible delisting
No data found for EVE-UN, possible delisting
No data found for EVE-WT, possible delisting
No data found for AEVA-WT, possible delisting
No data found for OASPW, possible delisting
No data found for NRXPW, possible delisting
No data found for ABLVW, possible delisting


MDLVY: No timezone found, symbol may be delisted


No data found for MDLVY, possible delisting


SAAYY: No timezone found, symbol may be delisted


No data found for SAAYY, possible delisting


### Get Dividends

In [39]:
def fetch_and_store_dividend_data(db):
    yf_col = db["dividends"]
    count = 0
    sleep_time = 2.5
    for t in tickers:
        try:
            df = yf.Ticker(t).get_dividends(
               proxy=PROXY_SERVER
            )
            if df.empty:
                print(f"No data found for {t}")
                continue
            data_dict = df.reset_index().to_dict(orient='records')
            
            for record in data_dict:
                ts = str(record['Date']).replace('-', '')
                record['_id'] = f'{t}_DV_{ts[:8]}'
            
            yf_col.insert_many(data_dict)
            count += 1
        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                if sleep_time<3.0:
                    sleep_time += 0.5
                    print("429 error detected, delaying to respect rate limit...")
                time.sleep(10.0)
        time.sleep(sleep_time)
    print(f'{count} distinct tickers inserted')



In [40]:

fetch_and_store_dividend_data(db)

No data found for GOOGL
No data found for AMZN
No data found for BRK-B
No data found for TSLA
No data found for AMD
No data found for NFLX
No data found for PDD
No data found for NOW
No data found for UBER
No data found for ISRG
No data found for BSX
No data found for ARM
No data found for TOELY
No data found for VRTX
No data found for ABNB
No data found for REGN
No data found for PANW
No data found for SHOP
No data found for FI
No data found for CMG
No data found for ANET
No data found for SNPS
No data found for SHECY
No data found for CDNS
No data found for SMFG
No data found for PYPL
No data found for CRWD
No data found for WDAY
No data found for ORLY
No data found for SPOT
No data found for GLD
No data found for DASH
No data found for SNOW
No data found for CPRT
No data found for NU
No data found for AZO
No data found for EW
No data found for DXCM
No data found for SMCI
No data found for COIN
No data found for PLTR
No data found for MFG
No data found for FTNT
No data found for LULU

JMKJ: No timezone found, symbol may be delisted


No data found for JMKJ
No data found for ABCL
No data found for VITL
No data found for DGII
No data found for TH
No data found for CARS
No data found for PRTA
No data found for CIFR
No data found for GOOS
No data found for THR
No data found for AMRC
No data found for ANGI
No data found for CRON
No data found for ENVX
No data found for BV
No data found for BLBD
No data found for GENI
No data found for SCSC
No data found for TDCX
No data found for CSIQ
No data found for CNTA
No data found for ZIP
No data found for TASK
No data found for NN
No data found for SHCO
No data found for IMTX
No data found for HUYA
No data found for PRM
No data found for CGC
No data found for ABVX
No data found for SP
No data found for ROOT
No data found for QNST
No data found for EAI
No data found for TRNS
No data found for AUTL
No data found for NG
No data found for REX
No data found for RPAY
No data found for NRDS
No data found for ARQT
No data found for GTI
No data found for ALHC
No data found for CVII
No da

ETH: No timezone found, symbol may be delisted


No data found for ETH
No data found for VREX
No data found for AMBC
No data found for ANAB
No data found for TMCI
No data found for BTDR
No data found for AZUL
No data found for LAZR
No data found for KYTX
No data found for SVA
No data found for DRQ
No data found for MTC
No data found for PNTG
No data found for EXK
No data found for SVRA
No data found for COGT
No data found for DNTH
No data found for AOSL
No data found for VMEO
No data found for NBBK
No data found for HKD
No data found for SOC
No data found for PGRU
No data found for BH-A
No data found for EXAI
No data found for BLND
No data found for AIRJ
No data found for QTRX
No data found for ACCD
No data found for XPOF
No data found for NVAX
No data found for EVLV
No data found for IREN
No data found for RUPRF
No data found for ORIC
No data found for PACK
No data found for SNDL
No data found for CCRN
No data found for LPRO
No data found for BITF
No data found for FLGT
No data found for LZM
No data found for NR
No data found for AS

...

No data found for TTNPW
No data found for CMCAU
No data found for CMCAW
No data found for EVE-UN
No data found for EVE-WT
No data found for AEVA-WT
No data found for OASPW
No data found for NRXPW
No data found for ABLVW


MDLVY: No timezone found, symbol may be delisted


No data found for MDLVY


SAAYY: No timezone found, symbol may be delisted


No data found for SAAYY
4296 distinct tickers inserted


### Get Splits

In [ ]:
def fetch_and_store_split_data(db):
    yf_col = db["splits"]
    for t in tickers:
        try:
            df = yf.Ticker(t).get_splits(proxy=PROXY_SERVER)
            if df.empty:
                print(f"No data found for {t}, possible delisting")
                continue
            data_dict = df.reset_index().to_dict(orient="records")

            for record in data_dict:
                ts = str(record["Date"]).replace("-", "")
                record["_id"] = f"{t}_SPLT_{ts[:8]}"

            yf_col.insert_many(data_dict)
        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                if sleep_time < 3.0:
                    sleep_time += 0.5
                    print("429 error detected, delaying to respect rate limit...")
                time.sleep(10.0)
        time.sleep(sleep_time)

In [ ]:
fetch_and_store_split_data(db)

### Get Balance Sheet

In [82]:
def fetch_and_store_balance_sheet_data(db):
    yf_col = db["balance_sheets"]
    sleep_time = 2.5

    
    for t in tickers:
        try:
            ticker_obj = yf.Ticker(t)
            df = ticker_obj.get_balance_sheet(proxy=PROXY_SERVER)
            if df.empty:
                print(f"No data found for {t}, possible delisting")
                continue

            # Reset index to work with columns easily
            df = df.reset_index()
            df.columns = ['Metric'] + [x.strftime('%Y%m%d') for x in df.columns[1:]]  # Convert Timestamps to string
            
            data_dicts = []
            for _, row in df.iterrows():
                for date in df.columns[1:]:
                    data_dict = {
                        # '_id': f'{t}_BAL_{date}',  # Unique ID per ticker, metric, and date
                        'ticker': t,
                        'date': date,
                        'metric': row['Metric'],
                        'value': row[date]
                    }
                    data_dicts.append(data_dict)
            
            yf_col.insert_many(data_dicts)
        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                if sleep_time<3.0:
                    sleep_time += 0.5
                    print("429 error detected, delaying to respect rate limit...")
                time.sleep(10.0)
        time.sleep(sleep_time)


In [83]:
start_time = time.time()
fetch_and_store_balance_sheet_data(db) 
end_time = time.time()
total_time = end_time - start_time
print(f"Program ran for {total_time:.2f} seconds")

No data found for AVBH, possible delisting
No data found for DBE, possible delisting
No data found for FXY, possible delisting
No data found for LION, possible delisting
No data found for EAXR, possible delisting
No data found for BMBN, possible delisting
No data found for RFM, possible delisting
No data found for FOTB, possible delisting
No data found for BAYA, possible delisting
No data found for DWNX, possible delisting
No data found for REDW, possible delisting
No data found for AMIX, possible delisting
No data found for GBUX, possible delisting
No data found for KSBI, possible delisting
No data found for BBUZ, possible delisting
No data found for TECTP, possible delisting
No data found for CPBI, possible delisting
No data found for WBQNL, possible delisting
No data found for FCCI, possible delisting
No data found for PCSV, possible delisting
No data found for CWPE, possible delisting
No data found for SLDX, possible delisting
No data found for PCST, possible delisting
No data foun

KeyboardInterrupt: 

### Get Cashflow

In [109]:
def fetch_and_store_cashflow_data(db):
    yf_col = db["cashflow"]
    sleep_time = 2.5

    for t in tickers:
        try:
            # Fetch cash flow data
            df = yf.Ticker(t).get_cashflow(proxy=PROXY_SERVER)
            if df.empty:
                print(f"No data found for {t}, possible delisting")
                continue

            # Reset index to work with columns easily
            df = df.reset_index()
            # Ensure datetime columns are formatted as string 'YYYYMMDD'
            df.columns = ['Metric'] + [str(col.date()) for col in df.columns[1:]]
            
            # Iterate over each date to create a separate document for each date
            for date in df.columns[1:]:
                data_dict = {'ticker': t, 'date': date}
                for _, row in df.iterrows():
                    data_dict[row['Metric']] = row[date]
                yf_col.insert_one(data_dict)

        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                if sleep_time<3.0:
                    sleep_time += 0.5
                    print("429 error detected, delaying to respect rate limit...")
                time.sleep(10.0)
        time.sleep(sleep_time)

In [110]:
fetch_and_store_cashflow_data(db)


No data found for SPY, possible delisting
No data found for QQQ, possible delisting
No data found for GLD, possible delisting
No data found for IAU, possible delisting
No data found for GBTC, possible delisting
No data found for DIA, possible delisting
No data found for MDY, possible delisting
No data found for USO, possible delisting
No data found for SLV, possible delisting
No data found for RBRK, possible delisting
No data found for DBC, possible delisting
No data found for EXG, possible delisting
No data found for ADX, possible delisting
No data found for ETY, possible delisting
No data found for USA, possible delisting
No data found for NUV, possible delisting
No data found for CLM, possible delisting
No data found for TY, possible delisting
No data found for BMEZ, possible delisting
No data found for PDO, possible delisting
No data found for ETV, possible delisting
No data found for GSG, possible delisting
No data found for BXMX, possible delisting
No data found for NFJ, possible

### Get Option Chain

In [135]:
def fetch_and_store_option_chain_data(db):
    yf_col = db["options"]
    sleep_time = 2.5
    count = 0
    for t in tickers:
        try:
            options = yf.Ticker(t).option_chain()
            calls = options.calls
            puts = options.puts

            # Process each call and put option
            for _, call in calls.iterrows():
                call_data = {
                    'ticker': t,
                    'type': 'call',
                    'contractSymbol': call['contractSymbol'],
                    'lastTradeDate': call['lastTradeDate'],
                    'strike': call['strike'],
                    'lastPrice': call['lastPrice'],
                    'bid': call['bid'],
                    'ask': call['ask'],
                    'change': call['change'],
                    'percentChange': call['percentChange'],
                    'volume': call['volume'],
                    'openInterest': call['openInterest'],
                    'impliedVolatility': call['impliedVolatility'],
                    'inTheMoney': call['inTheMoney'],
                    'contractSize': call['contractSize'],
                    'currency': call['currency']
                }
                yf_col.insert_one(call_data)  # Insert each call into MongoDB

            for _, put in puts.iterrows():
                put_data = {
                    'ticker': t,
                    'type': 'put',
                    'contractSymbol': put['contractSymbol'],
                    'lastTradeDate': put['lastTradeDate'],
                    'strike': put['strike'],
                    'lastPrice': put['lastPrice'],
                    'bid': put['bid'],
                    'ask': put['ask'],
                    'change': put['change'],
                    'percentChange': put['percentChange'],
                    'volume': put['volume'],
                    'openInterest': put['openInterest'],
                    'impliedVolatility': put['impliedVolatility'],
                    'inTheMoney': put['inTheMoney'],
                    'contractSize': put['contractSize'],
                    'currency': put['currency']
                }
                yf_col.insert_one(put_data)  # Insert each put into MongoDB
            count += 1

        except Exception as e:
            print(f"Error fetching data for {t}: {str(e)}")
            if "429" in str(e):
                return
        time.sleep(sleep_time)
    print(f'{count} distinct tickers inserted')


In [136]:
fetch_and_store_option_chain_data(db)


Error fetching data for LTMAY: 'calls'
Error fetching data for RTNTF: 'calls'
Error fetching data for EADSY: 'calls'
Error fetching data for AIQUY: 'calls'
Error fetching data for TOELY: 'calls'
Error fetching data for ABBNY: 'calls'
Error fetching data for CSLLY: 'calls'
Error fetching data for ATLKY: 'calls'
Error fetching data for BNPQY: 'calls'
Error fetching data for CFRUY: 'calls'
Error fetching data for SHECY: 'calls'
Error fetching data for GLNCY: 'calls'
Error fetching data for PXD: 'calls'
Error fetching data for ANZGY: 'calls'
Error fetching data for BAESY: 'calls'
Error fetching data for DNZOY: 'calls'
Error fetching data for CTA-PA: 'calls'
Error fetching data for CRARY: 'calls'
Error fetching data for OLCLY: 'calls'
Error fetching data for CODYY: 'calls'
Error fetching data for DKILY: 'calls'
Error fetching data for PPERY: 'calls'
Error fetching data for GBTC: 'calls'
Error fetching data for PTCAY: 'calls'
Error fetching data for WTKWY: 'calls'
Error fetching data for MRA

In [9]:
df = yf.Ticker('BTC-USD').history(
    period="max",
    prepost=True,
    actions=True,
    proxy=PROXY_SERVER,
    keepna=True
            )

In [10]:
print(df)

                                   Open          High           Low  \
Date                                                                  
2014-09-17 00:00:00+00:00    465.864014    468.174011    452.421997   
2014-09-18 00:00:00+00:00    456.859985    456.859985    413.104004   
2014-09-19 00:00:00+00:00    424.102997    427.834991    384.532013   
2014-09-20 00:00:00+00:00    394.673004    423.295990    389.882996   
2014-09-21 00:00:00+00:00    408.084991    412.425995    393.181000   
...                                 ...           ...           ...   
2024-05-10 00:00:00+00:00  63055.191406  63446.742188  60208.781250   
2024-05-11 00:00:00+00:00  60793.355469  61451.152344  60492.625000   
2024-05-12 00:00:00+00:00  60793.503906  61818.156250  60632.601562   
2024-05-13 00:00:00+00:00           NaN           NaN           NaN   
2024-05-14 00:00:00+00:00  62914.046875  63056.210938  62844.414062   

                                  Close       Volume  Dividends  Stock Split